In [1]:
import math
import numpy as np
import google.protobuf
import sentencepiece

In [21]:
import pandas as pd
from collections import Counter
from collections import defaultdict
import copy
from tqdm import tqdm

import re
import math

import ast
from itertools import combinations
from numpy.linalg import det

pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None) 

In [22]:
temp_df_oqwen_mmlu = pd.read_csv('/u/home/t/toz015/Agent/mmlu_policy_df_oqwen_7B.csv')
temp_df_llama2_mmlu = pd.read_csv('/u/home/t/toz015/Agent/mmlu_policy_df_deepseek_llm_7b.csv')
temp_df_qwen_mmlu = pd.read_csv('/u/home/t/toz015/Agent/mmlu_policy_df_deepseek_qwen_7B.csv')



temp_df_oqwen = pd.read_csv('/u/home/t/toz015/Agent/arc_policy_df_oqwen_7B.csv')
temp_df_llama2 = pd.read_csv('/u/home/t/toz015/Agent/arc_policy_df_deepseek_llm_7b.csv')
temp_df_qwen = pd.read_csv('/u/home/t/toz015/Agent/arc_policy_df_deepseek_qwen_7B.csv')

    
temp_df_oqwen_arc_easy = pd.read_csv('/u/home/t/toz015/Agent/arc_policy_df_easy_oqwen_7B.csv')
temp_df_llama2_arc_easy = pd.read_csv('/u/home/t/toz015/Agent/arc_policy_df_easy_deepseek_llm_7b.csv')
temp_df_qwen_arc_easy = pd.read_csv('/u/home/t/toz015/Agent/arc_policy_df_easy_deepseek_qwen_7B.csv')



temp_df_oqwen_gpqa = pd.read_csv('/u/home/t/toz015/Agent/gpqa_policy_df_oqwen_7B.csv')
temp_df_llama2_gpqa = pd.read_csv('/u/home/t/toz015/Agent/gpqa_policy_df_deepseek_llm_7b.csv')
temp_df_qwen_gpqa = pd.read_csv('/u/home/t/toz015/Agent/gpqa_policy_df_deepseek_qwen_7B.csv')

# temp_df_oqwen_mmlu = pd.read_csv('Data/mmlu_policy_df_oqwen_7B.csv')
# temp_df_llama2_mmlu = pd.read_csv('Data/mmlu_policy_df_deepseek_llm_7b.csv')
# temp_df_qwen_mmlu = pd.read_csv('Data/mmlu_policy_df_deepseek_qwen_7B.csv')



# temp_df_oqwen = pd.read_csv('Data/arc_policy_df_oqwen_7B.csv')
# temp_df_llama2 = pd.read_csv('Data/arc_policy_df_deepseek_llm_7b.csv')
# temp_df_qwen = pd.read_csv('Data/arc_policy_df_deepseek_qwen_7B.csv')

    
# temp_df_oqwen_arc_easy = pd.read_csv('Data/arc_policy_df_easy_oqwen_7B.csv')
# temp_df_llama2_arc_easy = pd.read_csv('Data/arc_policy_df_easy_deepseek_llm_7b.csv')
# temp_df_qwen_arc_easy = pd.read_csv('Data/arc_policy_df_easy_deepseek_qwen_7B.csv')



# temp_df_oqwen_gpqa = pd.read_csv('Data/gpqa_policy_df_oqwen_7B.csv')
# temp_df_llama2_gpqa = pd.read_csv('Data/gpqa_policy_df_deepseek_llm_7b.csv')
# temp_df_qwen_gpqa = pd.read_csv('Data/gpqa_policy_df_deepseek_qwen_7B.csv')

    

In [23]:
temp_df_qwen_gpqa.columns

Index(['question', 'subject', 'choices', 'correct_index', 'correct_letter',
       'gen_init_answer', 'disc_answer', 'gen_answer', 'disc_init_answer',
       'disc_final_policy_consensus', 'disc_init_policy', 'gen_init_policy',
       'gen_final_policy_consensus'],
      dtype='object')

In [24]:

temp_df_qwen["answer_letter"] = temp_df_qwen["answerKey"]
temp_df_oqwen["answer_letter"] = temp_df_oqwen["answerKey"]
temp_df_llama2["answer_letter"] = temp_df_llama2["answerKey"]



temp_df_qwen_arc_easy["answer_letter"] = temp_df_qwen_arc_easy["answerKey"]
temp_df_oqwen_arc_easy["answer_letter"] = temp_df_oqwen_arc_easy["answerKey"]
temp_df_llama2_arc_easy["answer_letter"] = temp_df_llama2_arc_easy["answerKey"]



dict_map = {0:"A", 1:"B", 2: "C", 3: "D"}
temp_df_qwen_mmlu["answer_letter"] = temp_df_qwen_mmlu["answer"].map(dict_map)
temp_df_oqwen_mmlu["answer_letter"] = temp_df_oqwen_mmlu["answer"].map(dict_map)
temp_df_llama2_mmlu["answer_letter"] = temp_df_llama2_mmlu["answer"].map(dict_map)


temp_df_qwen_gpqa["answer_letter"] = temp_df_qwen_gpqa["correct_letter"]
temp_df_oqwen_gpqa["answer_letter"] = temp_df_oqwen_gpqa["correct_letter"]
temp_df_llama2_gpqa["answer_letter"] = temp_df_llama2_gpqa["correct_letter"]


In [25]:
def evaluate_answer_alignment(df_qwen, df_oqwen, df_llama2, answer_col='disc_init_answer', label_col='answer_letter'):
    """
    Compare predicted answers and ground truth across three DataFrames.
    
    Args:
        df_qwen (DataFrame): First model's dataframe (e.g., Qwen).
        df_oqwen (DataFrame): Second model's dataframe (e.g., Oqwen).
        df_llama2 (DataFrame): Third model's dataframe (e.g., Llama2).
        answer_col (str): Name of the model prediction column (default 'disc_init_answer').
        label_col (str): Name of the ground truth column (default 'answer_letter').
        
    Returns:
        DataFrame: Table summarizing counts and accuracies.
    """
    
    total_n = df_oqwen.shape[0]
    
    qwen_corr = (df_qwen[answer_col] == df_qwen[label_col]).sum()
    oqwen_corr = (df_oqwen[answer_col] == df_oqwen[label_col]).sum()
    llama2_corr = (df_llama2[answer_col] == df_llama2[label_col]).sum()

    qwen_llama2_oqwen = ((df_oqwen[answer_col] == df_llama2[answer_col]) & (df_llama2[answer_col] == df_llama2[label_col]) & (df_qwen[answer_col] == df_oqwen[answer_col])).sum()
    
    results_d = {
        "Metric": [
            "Total Question #",
            "oqwen correct answers",
            "qwen correct answers",
            "llama2 correct answers",
            "qwen llama2 oqwen", 
        ],
        "Count": [
            total_n,
            oqwen_corr,
            qwen_corr,
            llama2_corr,
            qwen_llama2_oqwen,
        ]  
    }
    
    results_df = pd.DataFrame(results_d)
    results_df["Accuracy"] = (results_df["Count"] / total_n).apply(lambda x: f"{x:.0%}")
    
    return results_df


In [26]:

COMMON_COLUMNS = ["disc_init_policy", "gen_init_policy", "answer_letter", "disc_answer", "gen_answer"]
RENAME_MAPPING = {"disc_answer": "ED_consensus", "gen_answer": "EG_consensus"}

def process_dataframe(temp_df):
    """Process a dataframe by selecting columns and renaming them."""
    df = temp_df[COMMON_COLUMNS].copy()
    df.rename(columns=RENAME_MAPPING, inplace=True)
    return df

# ARC Challenge datasets
df_oqwen = process_dataframe(temp_df_oqwen)
df_deepseekqwen = process_dataframe(temp_df_qwen)
df_deepseekllama = process_dataframe(temp_df_llama2)

# ARC Easy datasets
df_oqwen_arc_easy = process_dataframe(temp_df_oqwen_arc_easy)
df_deepseekqwen_arc_easy = process_dataframe(temp_df_qwen_arc_easy)
df_deepseekllama_arc_easy = process_dataframe(temp_df_llama2_arc_easy)

# MMLU datasets
df_oqwen_mmlu = process_dataframe(temp_df_oqwen_mmlu)
df_deepseekqwen_mmlu = process_dataframe(temp_df_qwen_mmlu)
df_deepseekllama_mmlu = process_dataframe(temp_df_llama2_mmlu)

# GPQA datasets
df_oqwen_gpqa = process_dataframe(temp_df_oqwen_gpqa)
df_deepseekqwen_gpqa = process_dataframe(temp_df_qwen_gpqa)
df_deepseekllama_gpqa = process_dataframe(temp_df_llama2_gpqa)

In [27]:

for df in [
    df_oqwen, df_deepseekqwen, df_deepseekllama,
    df_oqwen_mmlu, df_deepseekqwen_mmlu, df_deepseekllama_mmlu,
    df_oqwen_arc_easy, df_deepseekqwen_arc_easy, df_deepseekllama_arc_easy,
    df_oqwen_gpqa, df_deepseekqwen_gpqa, df_deepseekllama_gpqa
]:
    df["disc_init_policy"] = df["disc_init_policy"].apply(ast.literal_eval)
    df["gen_init_policy"] = df["gen_init_policy"].apply(ast.literal_eval)

In [28]:
#Normalized the incorrect and correct policy in determinator
def apply_softmax(choice_dict):
    # Extract correct and incorrect values for all choices
    
    correct_values = np.array([v['correct'] for v in choice_dict.values()])
    incorrect_values = np.array([v['incorrect'] for v in choice_dict.values()])
    
    # Define softmax function
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()
    
    softmax_correct = softmax(correct_values)
    softmax_incorrect = softmax(incorrect_values)
    
    result = {}
    for i, choice in enumerate(choice_dict.keys()):
        result[choice] = {
            'correct': softmax_correct[i],
            'incorrect': softmax_incorrect[i]
        }
    return result



for df in [
    df_oqwen, df_deepseekqwen, df_deepseekllama,
    df_oqwen_mmlu, df_deepseekqwen_mmlu, df_deepseekllama_mmlu,
    df_oqwen_arc_easy, df_deepseekqwen_arc_easy, df_deepseekllama_arc_easy,
    df_oqwen_gpqa, df_deepseekqwen_gpqa, df_deepseekllama_gpqa
]:
    df["disc_init_policy_refine"] = df["disc_init_policy"].apply(apply_softmax)

    

In [29]:
def get_most_probable_letter(answer_dict):
    
    if answer_dict is None:
        return None 
    
    letter, _ = max(answer_dict.items(), key=lambda item: item[1]['correct'])
    return letter

for df in [
    df_oqwen, df_deepseekqwen, df_deepseekllama,
    df_oqwen_mmlu, df_deepseekqwen_mmlu, df_deepseekllama_mmlu,
    df_oqwen_arc_easy, df_deepseekqwen_arc_easy, df_deepseekllama_arc_easy,
    df_oqwen_gpqa, df_deepseekqwen_gpqa, df_deepseekllama_gpqa
]:
    df["disc_init_answer"] = df["disc_init_policy"].apply(get_most_probable_letter)
    df["disc_init_refine_answer"] = df["disc_init_policy_refine"].apply(get_most_probable_letter)
    


In [30]:
def reweight_joint(row):
    plm_y_given_xl = row['gen_init_policy'] 
    plm_l_given_xy = row['disc_init_policy']  

    new_joint = {}
    for y, label_probs in plm_y_given_xl.items():
        new_joint[y] = {}
        for label, p1 in label_probs.items():
            p2 = plm_l_given_xy.get(label, {}).get(y, 0.0)
            new_joint[y][label] = p1 * p2
    return new_joint


def get_max_correct_label(mi_dict):
    return max(mi_dict['correct'], key=mi_dict['correct'].get)


for df in [
    df_oqwen, df_deepseekqwen, df_deepseekllama,
    df_oqwen_mmlu, df_deepseekqwen_mmlu, df_deepseekllama_mmlu,
    df_oqwen_arc_easy, df_deepseekqwen_arc_easy, df_deepseekllama_arc_easy,
    df_oqwen_gpqa, df_deepseekqwen_gpqa, df_deepseekllama_gpqa
]:
    df["MI"] = df.apply(reweight_joint, axis=1)
    df["MI_answer"] = df["MI"].apply(get_max_correct_label)

    

In [31]:


def evaluate_majority_vote_fixed(df_qwen, df_oqwen, df_llama2, answer_col='disc_init_refine_answer', label_col='answer_letter'):
    """
    Compare three models' predictions with ground truth, and compute majority vote accuracy.
    This function recalculates model correctness based on combined_df only.
    
    Args:
        df_qwen (DataFrame): First model's dataframe (e.g., Qwen).
        df_oqwen (DataFrame): Second model's dataframe (e.g., OQwen).
        df_llama2 (DataFrame): Third model's dataframe (e.g., Llama2).
        answer_col (str): Name of the model prediction column (default 'disc_init_refine_answer').
        label_col (str): Name of the ground truth column (default 'answer_letter').
        
    Returns:
        combined_df (DataFrame): DataFrame containing all predictions and majority votes.
        results_df (DataFrame): Summary table of counts and accuracies.
    """

    # Combine predictions
    combined_df = pd.DataFrame({
        "question_id": df_oqwen.index,
        "correct_answer": df_oqwen[label_col],   # Use OQwen's label (or consistent label)
        "qwen_pred": df_qwen[answer_col],
        "oqwen_pred": df_oqwen[answer_col],
        "llama2_pred": df_llama2[answer_col]
    })
    
    # Majority vote
    def get_majority_vote(row):
        votes = [row["qwen_pred"], row["oqwen_pred"], row["llama2_pred"]]
        vote_counts = Counter(votes)
        majority_answers = [ans for ans, count in vote_counts.items() if count >= 2]
        if not majority_answers:
            return None
        return majority_answers[0]
    
    combined_df["majority_vote"] = combined_df.apply(get_majority_vote, axis=1)
    
    # Check correctness
    combined_df["qwen_correct"] = combined_df["qwen_pred"] == combined_df["correct_answer"]
    combined_df["oqwen_correct"] = combined_df["oqwen_pred"] == combined_df["correct_answer"]
    combined_df["llama2_correct"] = combined_df["llama2_pred"] == combined_df["correct_answer"]
    combined_df["is_majority_correct"] = combined_df["majority_vote"] == combined_df["correct_answer"]

    # Summarize
    total_questions = len(combined_df)
    qwen_corr = combined_df["qwen_correct"].sum()
    oqwen_corr = combined_df["oqwen_correct"].sum()
    llama2_corr = combined_df["llama2_correct"].sum()
    majority_correct = combined_df["is_majority_correct"].sum()

    results_d = {
        "Metric": [
            "Total Questions",
            "Qwen Accuracy",
            "OQwen Accuracy",
            "Llama2 Accuracy",
            "Majority Vote (≥2 models) Accuracy",
        ],
        "Count": [
            total_questions,
            qwen_corr,
            oqwen_corr,
            llama2_corr,
            majority_correct,
        ],
        "Accuracy": [
            "100%",
            f"{(qwen_corr / total_questions) * 100:.2f}%",
            f"{(oqwen_corr / total_questions) * 100:.2f}%",
            f"{(llama2_corr / total_questions) * 100:.2f}%",
            f"{(majority_correct / total_questions) * 100:.2f}%",
        ]
    }

    results_df = pd.DataFrame(results_d)
    
    return combined_df, results_df


In [60]:
np.random.seed(42)

class Discriminator:
    def __init__(self, init_policy):
        """Initialize with given probability dictionary."""
        self.policy = init_policy 

    def respond(self, choice, max_choice):
        """Sample a response ('correct' or 'incorrect') for a given answer choice."""
        prob_correct = self.policy[choice]['correct']
        res = "correct" if choice == max_choice else "incorrect"    
#         if choice == max_choice:
#             res = "correct"
#         else:
#             res = np.random.choice(["correct", "incorrect"], p=[prob_correct, 1- prob_correct])
        
        return res

    def log_gradient(self, choice, response):

        if response == "correct":
            grad_correct = 1 / self.policy[choice]['correct']
            grad_incorrect = -1 / self.policy[choice]['correct']
        else:
            grad_correct = -1 / self.policy[choice]['incorrect']
            grad_incorrect = 1 / self.policy[choice]['incorrect']

        return {'correct': grad_correct, 'incorrect': grad_incorrect}

    def get_max_correct_choice(self):
        """Return the choice with highest 'correct' probability"""
        return max(self.policy.items(), key=lambda x: x[1]['correct'])[0]
    
    def update_policy(self, choice, response, reward, learning_rate=0.1):
        grads = self.log_gradient(choice, response)
        prob_correct = self.policy[choice]['correct']
        prob_incorrect = self.policy[choice]['incorrect']
        
        log_prob_correct = np.log(prob_correct) + learning_rate * reward * grads['correct']
        log_prob_incorrect = np.log(prob_incorrect) + learning_rate * reward * grads['incorrect']

        max_log = max(log_prob_correct, log_prob_incorrect)
        log_prob_correct -= max_log
        log_prob_incorrect -= max_log

        exp_correct = np.exp(log_prob_correct)
        exp_incorrect = np.exp(log_prob_incorrect)
        total = exp_correct + exp_incorrect
        self.policy[choice]['correct'] = exp_correct / total
        self.policy[choice]['incorrect'] = exp_incorrect / total
        
        self.policy[choice]['correct'] = np.clip(self.policy[choice]['correct'], 1e-6, 1 - 1e-6)
        self.policy[choice]['incorrect'] = 1 - self.policy[choice]['correct']




label_to_index = {'correct': 0, 'incorrect': 1}

def normalize_rewards(dmi_dict):
    values = np.array(list(dmi_dict.values()))
    mean = np.mean(values)
    std = np.std(values) + 1e-8
    return {k: (v - mean) / std for k, v in dmi_dict.items()}


        
def generate_matrix(part_data, i, j):
    matrix = np.zeros((2, 2), dtype=int)
    for row in part_data:
        row_i = label_to_index[row[i]]
        row_j = label_to_index[row[j]]
        matrix[row_i][row_j] += 1
    return matrix


def calculate_dmi_score(part1_matrix, part2_matrix):
    det_part1 = np.linalg.det(part1_matrix)
    det_part2 = np.linalg.det(part2_matrix)
    dmi_score = det_part1 * det_part2
    return dmi_score

def compute_dmi_payments(responses):
    """Compute DMI payments using determinants of agreement matrices."""
    n = len(responses)
    split1 = responses[ : (n + 1)//2]  
    split2 = responses[(n + 1)//2 : ]  
    splits = {'Part1': split1, 'Part2': split2}
    
    n_d = len(responses[0])
    d_dmi_dict = {i : 0 for i in range(n_d)}
    for i, j in combinations(range(n_d), 2):
        part1_matrix = generate_matrix(splits["Part1"], i, j)
        part2_matrix = generate_matrix(splits["Part2"], i, j)
        dmi_score = calculate_dmi_score(part1_matrix, part2_matrix)
        d_dmi_dict[i] += dmi_score
        d_dmi_dict[j] += dmi_score
        
    return d_dmi_dict


def batch_update_discriminators_flexible(
    df_discriminator01, df_discriminator02, df_discriminator03,
    choice_df,
    batch_size=8,
    T_steps=10,
    learning_rate=0.1
):
    """
    Batch update discriminator policies for three dataframes.
    
    Args:
        df_discriminator01 (DataFrame): DataFrame containing 'disc_init_policy_refine' for OQwen.
        df_discriminator02 (DataFrame): DataFrame containing 'disc_init_policy_refine' for DeepseekQwen.
        df_discriminator03 (DataFrame): DataFrame containing 'disc_init_policy_refine' for DeepseekLlama.
        choice_df (DataFrame): DataFrame used for selecting initial choices (e.g., DeepseekQwen's 'gen_init_policy').
        batch_size (int): Number of rows to update in each batch.
        T_steps (int): Number of response-update iterations per batch.
        learning_rate (float): Learning rate for discriminator policy update.
        
    Returns:
        Updated DataFrames (inplace modification too).
    """
    if batch_size < 4:
        print("batch size should larger than 4")
    # Initialize updated_disc_policy columns
    for df in [df_discriminator01, df_discriminator02, df_discriminator03]:
        df["updated_disc_policy"] = None

    n_rows = df_discriminator01.shape[0]
    
    for i in range(0, n_rows - batch_size, batch_size):
        # Initialize discriminators
        discriminators = [
            [Discriminator(copy.deepcopy(df_discriminator01.loc[i + j, 'disc_init_policy_refine'])) for j in range(batch_size)],
            [Discriminator(copy.deepcopy(df_discriminator02.loc[i + j, 'disc_init_policy_refine'])) for j in range(batch_size)],
            [Discriminator(copy.deepcopy(df_discriminator03.loc[i + j, 'disc_init_policy_refine'])) for j in range(batch_size)]
        ]

        # Initialize choices based on the passed-in choice_df
        choices = [
            max(choice_df.loc[i + j, "gen_init_policy"]["correct"],
                key=choice_df.loc[i + j, "gen_init_policy"]["correct"].get)
            for j in range(batch_size)
        ]

        responses = []
        for _ in range(T_steps):
            batch_responses = []
            for j in range(batch_size):
                task_responses = []
                for d in range(3):
                    max_choice = discriminators[d][j].get_max_correct_choice()
                    task_responses.append(discriminators[d][j].respond(choices[j], max_choice))
                batch_responses.append(task_responses)
            responses = batch_responses

            dmi_scores = compute_dmi_payments(batch_responses)
            dmi_scores = normalize_rewards(dmi_scores)

            # Update discriminator policies
            for d in range(3):
                for j in range(batch_size):
                    choice = choices[j]
                    response = batch_responses[j][d]
                    reward = dmi_scores[d]
                    discriminators[d][j].update_policy(choice, response, reward, learning_rate)

        # Save updated policies
        for j in range(batch_size):
            df_discriminator01.at[i + j, "updated_disc_policy"] = discriminators[0][j].policy
            df_discriminator02.at[i + j, "updated_disc_policy"] = discriminators[1][j].policy
            df_discriminator03.at[i + j, "updated_disc_policy"] = discriminators[2][j].policy
    
    
    df_discriminator01['updated_answer_letter'] = df_discriminator01['updated_disc_policy'].apply(get_most_probable_letter)
    df_discriminator02['updated_answer_letter'] = df_discriminator02['updated_disc_policy'].apply(get_most_probable_letter)
    df_discriminator03['updated_answer_letter'] = df_discriminator03['updated_disc_policy'].apply(get_most_probable_letter)


    return df_discriminator01, df_discriminator02, df_discriminator03

 

In [61]:
combined_df_inital_gen, results_df_inital_gen = evaluate_majority_vote_fixed(temp_df_qwen, temp_df_oqwen, temp_df_llama2,  answer_col='gen_init_answer', label_col='answer_letter')
display(results_df_inital_gen)

,Metric,Count,Accuracy
0,Total Questions,1170,100%
1,Qwen Accuracy,820,70.09%
2,OQwen Accuracy,1020,87.18%
3,Llama2 Accuracy,725,61.97%
4,Majority Vote (≥2 models) Accuracy,901,77.01%


In [62]:
results_table = evaluate_answer_alignment(temp_df_qwen, temp_df_oqwen, temp_df_llama2)

print("ARC Challenge")
display(results_table)


ARC Challenge


,Metric,Count,Accuracy
0,Total Question #,1170,100%
1,oqwen correct answers,970,83%
2,qwen correct answers,669,57%
3,llama2 correct answers,700,60%
4,qwen llama2 oqwen,457,39%


In [63]:
## ARC Challenger Original answer 

combined_df_inital, results_df_inital = evaluate_majority_vote_fixed(df_deepseekqwen, df_oqwen, df_deepseekllama,  answer_col='disc_init_answer', label_col='answer_letter')
display(results_df_inital)



,Metric,Count,Accuracy
0,Total Questions,1170,100%
1,Qwen Accuracy,669,57.18%
2,OQwen Accuracy,970,82.91%
3,Llama2 Accuracy,700,59.83%
4,Majority Vote (≥2 models) Accuracy,827,70.68%


In [65]:
## ARC Challenger Mutual Information Ranking
combined_df_MI, results_df_MI = evaluate_majority_vote_fixed(df_deepseekqwen, df_oqwen,  df_deepseekllama,  answer_col='MI_answer', label_col='answer_letter')
display(results_df_MI)

,Metric,Count,Accuracy
0,Total Questions,1170,100%
1,Qwen Accuracy,822,70.26%
2,OQwen Accuracy,1027,87.78%
3,Llama2 Accuracy,748,63.93%
4,Majority Vote (≥2 models) Accuracy,913,78.03%


In [66]:
## ARC Challenger ED consensus

combined_df_ED, results_df_ED = evaluate_majority_vote_fixed(df_deepseekqwen, df_oqwen, df_deepseekllama,  answer_col='ED_consensus', label_col='answer_letter')
display(results_df_ED)


,Metric,Count,Accuracy
0,Total Questions,1170,100%
1,Qwen Accuracy,822,70.26%
2,OQwen Accuracy,1025,87.61%
3,Llama2 Accuracy,741,63.33%
4,Majority Vote (≥2 models) Accuracy,907,77.52%


In [67]:
## ARC Challenger Original Qwen as problem selector

df_oqwen, df_deepseekqwen, df_deepseekllama =  batch_update_discriminators_flexible(
    df_oqwen, 
    df_deepseekqwen, 
    df_deepseekllama,
    choice_df = df_oqwen, 
    batch_size=8,
    T_steps=10,
    learning_rate=0.1
)
       
combined_df_refine_update, results_df_refine_update = evaluate_majority_vote_fixed(df_deepseekqwen, df_oqwen, df_deepseekllama,  answer_col='updated_answer_letter', label_col='answer_letter')
display(results_df_refine_update)

,Metric,Count,Accuracy
0,Total Questions,1170,100%
1,Qwen Accuracy,1008,86.15%
2,OQwen Accuracy,995,85.04%
3,Llama2 Accuracy,1006,85.98%
4,Majority Vote (≥2 models) Accuracy,1018,87.01%


In [68]:


results_table_arc_easy = evaluate_answer_alignment(temp_df_qwen_arc_easy, temp_df_oqwen_arc_easy, temp_df_llama2_arc_easy)

print("ARC Easy")
display(results_table_arc_easy)

ARC Easy


,Metric,Count,Accuracy
0,Total Question #,2371,100%
1,oqwen correct answers,2144,90%
2,qwen correct answers,1671,70%
3,llama2 correct answers,1809,76%
4,qwen llama2 oqwen,1415,60%


In [69]:

combined_df_inital_gen, results_df_inital_gen = evaluate_majority_vote_fixed(temp_df_qwen_arc_easy, temp_df_oqwen_arc_easy, temp_df_llama2_arc_easy,  answer_col='gen_init_answer', label_col='answer_letter')
display(results_df_inital_gen)

,Metric,Count,Accuracy
0,Total Questions,2371,100%
1,Qwen Accuracy,1935,81.61%
2,OQwen Accuracy,2180,91.94%
3,Llama2 Accuracy,1831,77.22%
4,Majority Vote (≥2 models) Accuracy,2091,88.19%


In [70]:
## ARC Easy Original answer 

combined_df_inital_arc_easy, results_df_inital_arc_easy = evaluate_majority_vote_fixed(df_deepseekqwen_arc_easy, df_oqwen_arc_easy, df_deepseekllama_arc_easy,  answer_col='disc_init_answer', label_col='answer_letter')
display(results_df_inital_arc_easy)


,Metric,Count,Accuracy
0,Total Questions,2371,100%
1,Qwen Accuracy,1671,70.48%
2,OQwen Accuracy,2144,90.43%
3,Llama2 Accuracy,1809,76.30%
4,Majority Vote (≥2 models) Accuracy,1996,84.18%


In [72]:
## ARC Easy Mutual Information Ranking
combined_df_MI_arc_easy, results_df_MI_arc_easy = evaluate_majority_vote_fixed(df_deepseekqwen_arc_easy, df_oqwen_arc_easy, df_deepseekllama_arc_easy,  answer_col='MI_answer', label_col='answer_letter')
display(results_df_MI_arc_easy)


,Metric,Count,Accuracy
0,Total Questions,2371,100%
1,Qwen Accuracy,1942,81.91%
2,OQwen Accuracy,2190,92.37%
3,Llama2 Accuracy,1853,78.15%
4,Majority Vote (≥2 models) Accuracy,2101,88.61%


In [73]:
## ARC Easy Consensus ED answer

combined_df_ED_arc_easy, results_df_ED_arc_easy = evaluate_majority_vote_fixed(df_deepseekqwen_arc_easy, df_oqwen_arc_easy,  df_deepseekllama_arc_easy,  answer_col='ED_consensus', label_col='answer_letter')
display(results_df_ED_arc_easy)

,Metric,Count,Accuracy
0,Total Questions,2371,100%
1,Qwen Accuracy,1940,81.82%
2,OQwen Accuracy,2188,92.28%
3,Llama2 Accuracy,1845,77.82%
4,Majority Vote (≥2 models) Accuracy,2100,88.57%


In [74]:
## ARC Easy Original Qwen as problem selector

df_oqwen_arc_easy, df_deepseekqwen_arc_easy, df_deepseekllama_arc_easy =  batch_update_discriminators_flexible(
    df_oqwen_arc_easy, 
    df_deepseekqwen_arc_easy, 
    df_deepseekllama_arc_easy,
    choice_df = df_oqwen_arc_easy, 
    batch_size=8,
    T_steps=10,
    learning_rate=0.1
)
       
combined_df_refine_update_arc_easy, results_df_refine_update_arc_easy = evaluate_majority_vote_fixed( df_deepseekqwen_arc_easy, df_oqwen_arc_easy,df_deepseekllama_arc_easy,  answer_col='updated_answer_letter', label_col='answer_letter')
display(results_df_refine_update_arc_easy)

,Metric,Count,Accuracy
0,Total Questions,2371,100%
1,Qwen Accuracy,2175,91.73%
2,OQwen Accuracy,2155,90.89%
3,Llama2 Accuracy,2168,91.44%
4,Majority Vote (≥2 models) Accuracy,2176,91.78%


In [75]:
combined_df_inital_gen, results_df_inital_gen = evaluate_majority_vote_fixed(temp_df_qwen_mmlu, temp_df_oqwen_mmlu, temp_df_llama2_mmlu,  answer_col='gen_init_answer', label_col='answer_letter')
display(results_df_inital_gen)

,Metric,Count,Accuracy
0,Total Questions,13869,100%
1,Qwen Accuracy,7404,53.39%
2,OQwen Accuracy,9849,71.01%
3,Llama2 Accuracy,6978,50.31%
4,Majority Vote (≥2 models) Accuracy,8316,59.96%


In [76]:
results_table_mmlu = evaluate_answer_alignment(temp_df_qwen_mmlu, temp_df_oqwen_mmlu, temp_df_llama2_mmlu)

print("MMLU")
display(results_table_mmlu)


MMLU


,Metric,Count,Accuracy
0,Total Question #,13869,100%
1,oqwen correct answers,8698,63%
2,qwen correct answers,5878,42%
3,llama2 correct answers,6411,46%
4,qwen llama2 oqwen,2957,21%


In [77]:
## MMLU Original answer
combined_df_inital_mmlu, results_df_inital_mmlu = evaluate_majority_vote_fixed(df_deepseekqwen_mmlu, df_oqwen_mmlu, df_deepseekllama_mmlu,  answer_col='disc_init_answer', label_col='answer_letter')
display(results_df_inital_mmlu)

,Metric,Count,Accuracy
0,Total Questions,13869,100%
1,Qwen Accuracy,5878,42.38%
2,OQwen Accuracy,8698,62.72%
3,Llama2 Accuracy,6411,46.23%
4,Majority Vote (≥2 models) Accuracy,7038,50.75%


In [79]:
## MMLU Mutual Information Ranking
combined_df_MI_mmlu, results_df_MI_mmlu = evaluate_majority_vote_fixed(df_deepseekqwen_mmlu, df_oqwen_mmlu, df_deepseekllama_mmlu,  answer_col='MI_answer', label_col='answer_letter')
display(results_df_MI_mmlu)


,Metric,Count,Accuracy
0,Total Questions,13869,100%
1,Qwen Accuracy,7435,53.61%
2,OQwen Accuracy,9830,70.88%
3,Llama2 Accuracy,6949,50.10%
4,Majority Vote (≥2 models) Accuracy,8301,59.85%


In [80]:
## MMLU Consensus ED answer

combined_df_ED_mmlu, results_df_ED_mmlu = evaluate_majority_vote_fixed(df_deepseekqwen_mmlu, df_oqwen_mmlu, df_deepseekllama_mmlu,  answer_col='ED_consensus', label_col='answer_letter')
display(results_df_ED_mmlu)

,Metric,Count,Accuracy
0,Total Questions,13869,100%
1,Qwen Accuracy,7430,53.57%
2,OQwen Accuracy,9761,70.38%
3,Llama2 Accuracy,6966,50.23%
4,Majority Vote (≥2 models) Accuracy,8274,59.66%


In [81]:
## l Qwen as problem selector

df_oqwen_mmlu, df_deepseekqwen_mmlu, df_deepseekllama_mmlu =  batch_update_discriminators_flexible(
    df_oqwen_mmlu, 
    df_deepseekqwen_mmlu, 
    df_deepseekllama_mmlu,
    choice_df = df_oqwen_mmlu, 
    batch_size=8,
    T_steps=10,
    learning_rate=0.1
)
       
combined_df_refine_update_mmlu, results_df_refine_update_mmlu = evaluate_majority_vote_fixed(df_deepseekqwen_mmlu, df_oqwen_mmlu,  df_deepseekllama_mmlu,  answer_col='updated_answer_letter', label_col='answer_letter')
display(results_df_refine_update_mmlu)

,Metric,Count,Accuracy
0,Total Questions,13869,100%
1,Qwen Accuracy,9575,69.04%
2,OQwen Accuracy,9457,68.19%
3,Llama2 Accuracy,9530,68.71%
4,Majority Vote (≥2 models) Accuracy,9809,70.73%


In [82]:

combined_df_inital_gen, results_df_inital_gen = evaluate_majority_vote_fixed(temp_df_qwen_gpqa, temp_df_oqwen_gpqa, temp_df_llama2_gpqa,  answer_col='gen_init_answer', label_col='answer_letter')
display(results_df_inital_gen)

,Metric,Count,Accuracy
0,Total Questions,448,100%
1,Qwen Accuracy,82,18.30%
2,OQwen Accuracy,101,22.54%
3,Llama2 Accuracy,88,19.64%
4,Majority Vote (≥2 models) Accuracy,81,18.08%


In [83]:
## GPQA
combined_df_inital_gpqa, results_df_inital_gpqa = evaluate_majority_vote_fixed(df_deepseekqwen_gpqa, df_oqwen_gpqa,  df_deepseekllama_gpqa,  answer_col='disc_init_answer', label_col='answer_letter')
display(results_df_inital_gpqa)

,Metric,Count,Accuracy
0,Total Questions,448,100%
1,Qwen Accuracy,68,15.18%
2,OQwen Accuracy,72,16.07%
3,Llama2 Accuracy,78,17.41%
4,Majority Vote (≥2 models) Accuracy,41,9.15%


In [84]:
## GPQA Initial answer
combined_df_refine_inital_gpqa, results_df_refine_inital_gpqa = evaluate_majority_vote_fixed(df_deepseekqwen_gpqa, df_oqwen_gpqa, df_deepseekllama_gpqa,  answer_col='disc_init_refine_answer', label_col='answer_letter')
display(results_df_refine_inital_gpqa)


,Metric,Count,Accuracy
0,Total Questions,448,100%
1,Qwen Accuracy,68,15.18%
2,OQwen Accuracy,72,16.07%
3,Llama2 Accuracy,78,17.41%
4,Majority Vote (≥2 models) Accuracy,41,9.15%


In [85]:
## GPQA Mutual Information Ranking
combined_df_MI_gpqa, results_df_MI_gpqa = evaluate_majority_vote_fixed(df_deepseekqwen_gpqa, df_oqwen_gpqa, df_deepseekllama_gpqa,  answer_col='MI_answer', label_col='answer_letter')
display(results_df_MI_gpqa)


,Metric,Count,Accuracy
0,Total Questions,448,100%
1,Qwen Accuracy,83,18.53%
2,OQwen Accuracy,88,19.64%
3,Llama2 Accuracy,89,19.87%
4,Majority Vote (≥2 models) Accuracy,73,16.29%


In [86]:
## GPQA Consensus ED answer

combined_df_ED_gpqa, results_df_ED_gpqa = evaluate_majority_vote_fixed(df_deepseekqwen_gpqa, df_oqwen_gpqa,  df_deepseekllama_gpqa,  answer_col='ED_consensus', label_col='answer_letter')
display(results_df_ED_gpqa)

,Metric,Count,Accuracy
0,Total Questions,448,100%
1,Qwen Accuracy,84,18.75%
2,OQwen Accuracy,90,20.09%
3,Llama2 Accuracy,85,18.97%
4,Majority Vote (≥2 models) Accuracy,74,16.52%


In [87]:
## GPQA DMI Qwen as problem selector

df_oqwen_gpqa, df_deepseekqwen_gpqa, df_deepseekllama_gpqa =  batch_update_discriminators_flexible(
    df_oqwen_gpqa, 
    df_deepseekqwen_gpqa, 
    df_deepseekllama_gpqa,
    choice_df = df_oqwen_gpqa, 
    batch_size=8,
    T_steps=10,
    learning_rate=0.1
)
       
combined_df_refine_update_gpqa, results_df_refine_update_gpqa = evaluate_majority_vote_fixed(df_deepseekqwen_gpqa, df_oqwen_gpqa,  df_deepseekllama_gpqa,  answer_col='updated_answer_letter', label_col='answer_letter')
display(results_df_refine_update_gpqa)

,Metric,Count,Accuracy
0,Total Questions,448,100%
1,Qwen Accuracy,90,20.09%
2,OQwen Accuracy,102,22.77%
3,Llama2 Accuracy,106,23.66%
4,Majority Vote (≥2 models) Accuracy,101,22.54%
